## Deneme

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def create_tr_wiki_ds():
    # dosya path'lerini al
    # regular expressionu hazırla
    # dataset objesi oluştur (tablo, kolonlar: examples, titles, #characters)
    # sırayla dosyaları(splitleri) aç
        # dataset objesini/tabloyu doldurmaya başla
    # işlem bitince tr-wiki67.arrow ismi ile kaydet (datasetdict objesinde başka splitler olmasın tek bir tane olsun)
    pass

In [3]:
from datasets import DatasetDict, Dataset
import re

In [4]:
lens = list(map(len,["ali ata bak", "bak... !"]))
lens

[11, 8]

In [6]:
prefix = "data/tr_wiki67/trwiki-67."
file_paths = [ f"{prefix}{post_fix}.raw" for post_fix in ["test", "train", "val"] ]
file_paths

['data/tr_wiki67/trwiki-67.test.raw',
 'data/tr_wiki67/trwiki-67.train.raw',
 'data/tr_wiki67/trwiki-67.val.raw']

In [7]:
data_dict = {"examples":[], "titles":[], "length":[]}

# go for every file
for file in file_paths:
    
    # open file
    with open(file) as f:
        
        # read its content
        content = f.read()
        
        # with regular exp. find titles, examples(doc), and also measure length of examples
        examples = list(map(str.strip, re.split("== .* ==", content)[1:]))
        titles = list(re.findall("== .* ==", content))
        length = list(map(len, examples))

        # package these with dict object
        temp_data_dict = {"examples":examples, "titles":titles, "length":length}
    
    # concat dict objects from diffrent files into one 
    data_dict["examples"].extend(temp_data_dict["examples"])
    data_dict["titles"].extend(temp_data_dict["titles"])
    data_dict["length"].extend(temp_data_dict["length"])



In [50]:
# lets print
print(data_dict["titles"][9625])
print(data_dict["examples"][9625])
print(data_dict["length"][9625])

== Gülek Kalesi ==
Gülek Kalesi Mersin ilinde bir kale örenidir.
Yerleşim.
Kale Mersin ili Tarsus ilçesi kuzeyinde Toros Dağları üzerinde, Gülek Boğazına hakim bir noktada ve koordinatlarındadır. Kaleye Gülek beldesinden ayrılan 5 kilometrelik bir yolla ulaşılmaktadır. Deniz seviyesinden yüksekliği 1530 metre, denetlediği Gülek Boğazından yüksekliği ise 500 metredir. Tarsus’a olan uzaklığı 65 kilometre, Mersin’e olan uzaklığı ise 95 kilometredir.
Tarihi.
Kalenin yapılış tarihi bilinmese de, Kilikya Ermeni Krallığı döneminde yapıldığı veya esaslı bir onarımdan geçtiği anlaşılmaktadır. Bu devlete ait 1198-1199 dönemine tarihlenen bir taç giyme listesinde Gülek lordu Symbat’ın adı geçmektedir. Kalenin o dönemdeki adı ‘’Asakaliba’’ idi. Kale İç Anadolu ile Akdeniz Bölgesi arasındaki en önemli geçit olan Gülek Boğazını denetlemek amacıyla yapılmıştır. Ortaçağ'da bu boğazı geçenler kale komutanına ücret ödemek zorunda kalıyorlardı.
Bölge daha sonra Memluk Devleti ve Osmanlı İmparatorluğu ege

In [63]:
# after delete subtitles
print(deneme[9625])

Gülek Kalesi Mersin ilinde bir kale örenidir.
Kale Mersin ili Tarsus ilçesi kuzeyinde Toros Dağları üzerinde, Gülek Boğazına hakim bir noktada ve koordinatlarındadır. Kaleye Gülek beldesinden ayrılan 5 kilometrelik bir yolla ulaşılmaktadır. Deniz seviyesinden yüksekliği 1530 metre, denetlediği Gülek Boğazından yüksekliği ise 500 metredir. Tarsus’a olan uzaklığı 65 kilometre, Mersin’e olan uzaklığı ise 95 kilometredir.
Kalenin yapılış tarihi bilinmese de, Kilikya Ermeni Krallığı döneminde yapıldığı veya esaslı bir onarımdan geçtiği anlaşılmaktadır. Bu devlete ait 1198-1199 dönemine tarihlenen bir taç giyme listesinde Gülek lordu Symbat’ın adı geçmektedir. Kalenin o dönemdeki adı ‘’Asakaliba’’ idi. Kale İç Anadolu ile Akdeniz Bölgesi arasındaki en önemli geçit olan Gülek Boğazını denetlemek amacıyla yapılmıştır. Ortaçağ'da bu boğazı geçenler kale komutanına ücret ödemek zorunda kalıyorlardı.
Bölge daha sonra Memluk Devleti ve Osmanlı İmparatorluğu egemenliğinde kalmış, 19.yüzyılda Kavala

In [9]:
len(data_dict["examples"][99999])

1356

In [10]:
len(data_dict["examples"]), len(data_dict["titles"])

(394134, 394134)

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data_dict)
df.describe()

,length
count,394134.000000
mean,1280.671939
std,3259.907825
min,0.000000
25%,204.000000
50%,453.000000
75%,1136.000000
max,193505.000000


In [15]:
df["length"].sum() # ~~ 500_000_000 characters

504756354

In [12]:
(df["length"] == 0).sum() # number of zero values

16

In [13]:
mask = ~(df["length"] == 0) # ~ symbol makes True -> False, False -> True

In [14]:
df = df[mask]

# now there will be no example that has not text-characters
(df["length"] == 0).sum() 

0

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
example_sentence = "Türkiye-İran55 ilişkileri.55_55 - 987@"
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(example_sentence)
words

['Türkiye', 'İran55', 'ilişkileri', '55_55', '987']

In [61]:
def delete_subtitles(example):
    temp = []
    # split lines but also keep new line character
    for line in example.splitlines(True):
        if len(tokenizer.tokenize(line)) > 5:
            temp.append(line)
    
    if len(temp) != 0:
        return "".join(temp)
    else:
        return None

In [62]:
# in our dataset (wiki), some lines are actually subtitles, this can make our continuation of 
# sentences worse (we will get pair of sentences for NSP task)
# for this reason we need to go through all lines of examples and eliminate lines that are smaller than 5 words
deneme = df["examples"].map(delete_subtitles)

In [71]:
deneme[deneme.isnull()].head(10)

462     None
603     None
1013    None
2936    None
2993    None
3527    None
3871    None
5961    None
6990    None
7270    None
Name: examples, dtype: object

In [73]:
print(data_dict["examples"][3871])
print(data_dict["examples"][603])
print(data_dict["examples"][1013])
print(data_dict["examples"][7270])

Sıralamalar
İllere göre nüfus sayıları.
Atabey kavramı aşağıdaki anlamlarda kullanılabilir
Örümcek, bir eklembacaklı hayvan türüdür.
Ayrıca şu anlamlara da gelebilir


In [64]:
deneme.isnull().sum()

324

In [23]:
# lets convert this df object into dict again
data_dict = df.to_dict(orient="list")

In [74]:
def delete_subtitles(example):
    """
    will be used as a map function
    take every example-doc and delete if it has subtitles (if line count of word is < 5)
    """
    temp = []
    # split lines
    for line in example.splitlines(True):
        # if count of word is > 5 just keep this line, if not > 5, we will consider it as subtitle and not gonna keep it
        if len(tokenizer.tokenize(line)) > 5:
            temp.append(line)
    
    # in some cases there will be no lines that are bigger than 5
    # also some examples may not have any text at all! (data construction failure)
    if len(temp) != 0:
        return "".join(temp)
    else:
        return None
    

def create_tr_wiki_ds():
    """
    + there are 3 files (splits): trwiki-67.test.raw, trwiki-67.train.raw, trwiki-67.val.raw
    + this function goes for every file
    + finds titles and doc-examples
    + concat everyting into one dict object
    + there are problematic examples (no text(null) or some lines of examples are not actually paragraph
      but subtitles)
    + apply preprocess to handle these problems (delete subtitles and mask-filter-delete null examples)
    + measure length of examples and return as a new column
    + create dataset object and save it as .arrow format
    """
    prefix = "trwiki-67."
    file_paths = [ f"{prefix}{post_fix}.raw" for post_fix in ["test", "train", "val"] ]

    data_dict = {"examples":[], "titles":[]}

    # go for every file
    for file in file_paths:

        # open file
        with open(file) as f:

            # read its content
            content = f.read()

            # with regular exp. find titles, examples(doc)
            examples = list(map(str.strip, re.split("== .* ==", content)[1:]))
            titles = list(re.findall("== .* ==", content))

            # package these with dict object
            temp_data_dict = {"examples":examples, "titles":titles}

        # concat dict objects from diffrent files into one overall dict object
        data_dict["examples"].extend(temp_data_dict["examples"])
        data_dict["titles"].extend(temp_data_dict["titles"])

        # convert data_dict object into dataframe object
        df = pd.DataFrame(data_dict)
        # lets delete lines of examples that are subtitles
        df["examples"] = df["examples"].map(delete_subtitles)
        # lets also delete-mask df by null values (examples that does not contain text at all)
        mask = df["examples"].isnull()
        df = df[mask]

        # lets measure length of examples (by char)
        df["length"] = df["examples"].map(len)

        # create hf dataset and save to disk
        df = df.reset_index(drop=True)
        dataset = Dataset.from_pandas(df)
        dataset.save_to_disk("trwiki_dataset")


In [102]:
df.columns

Index(['examples', 'titles', 'length', 'deneme kolonu'], dtype='object')

In [104]:
df2 = df
df2.reset_index(drop=True, inplace=True)
Dataset.from_pandas(df2)

Dataset({
    features: ['examples', 'titles', 'length', 'deneme kolonu'],
    num_rows: 394118
})

In [92]:
datasetdeneme = Dataset.from_pandas(df)

In [95]:
datasetdeneme

Dataset({
    features: ['examples', 'titles', 'length', 'deneme kolonu', '__index_level_0__'],
    num_rows: 394118
})

In [94]:
datasetdeneme.save_to_disk("deneme")

Saving the dataset (0/2 shards):   0%|          | 0/394118 [00:00<?, ? examples/s]

In [96]:
from datasets import load_from_disk
aaaa = load_from_disk("deneme")

In [97]:
aaaa

Dataset({
    features: ['examples', 'titles', 'length', 'deneme kolonu', '__index_level_0__'],
    num_rows: 394118
})

In [98]:
aaaa.remove_columns(["__index_level_0__", "deneme kolonu"])

Dataset({
    features: ['examples', 'titles', 'length'],
    num_rows: 394118
})

### Örnek metin (dene.txt):
== NGC 1710 == 

NGC 1710, Yeni Genel Katalog'da yer alan bir galaksidir. Gökyüzünde Aslan takımyıldızı yönünde bulunur. E-S0 tipi bir merceksi, eliptik galaksidir. Amerikan astronom Francis Leavenworth tarafından 1885 yılında 66,04 cm (26 inç) çaplı mercekli tip bir teleskopla keşfedilmiştir.

== Şenol Gürşan == 

Şenol Gürşan, (d. 17 Ekim 1964, Pınarhisar, Kırklareli) Türk avukat ve siyasetçi.
İstanbul Üniversitesi Hukuk Fakültesi'ni bitirmiş ve serbest avukat olarak çalışmıştır. Kırklareli İlim Yayma Cemiyeti Kuruculuğu ve Başkanlığı görevlerinde bulunmuştur.
2009 yılında Adalet ve Kalkınma Partisi Kırklareli il yönetim kurulu üyesi olmuş, TBMM 24. dönem AK Parti Kırklareli milletvekili, Türkiye-Polonya Dostluk Grubu Başkanı ve TBMM KİT Komisyonu Sözcüsü olmuştur. Gelecek Partisi Kurucular Kurulu üyesi olup aynı zamanda partinin genel sekreteridir.
İyi düzeyde Almanca bilen Gürşan, evli ve 2 çocuk babasıdır.

== Bovenau == 

Bovenau Almanya'nın kuzeyinde Schleswig-Holstein eyaletinde, Rendsburg-Eckernförde iline bağlı bölge. Rendsburg'un 10 km. doğusunda yer alır. 26,2 km² yüzölçümüne sahiptir. Nüfusu, 17 Temmuz 2006 itibarıyla yaklaşık 1098 olarak tespit edilmiştir. Belediye Başkanlığı Jürgen Liebsch tarafından yürütülmektedir.

In [ ]:
import re

with open("trwiki-67.test.txt") as fi:
    content = fi.read()
    articles = [ { "title": ti.strip("==").strip(), 
                   "text": tx.strip()} 
                   for ti, tx in 
                   zip(re.findall("== .* ==", content), re.split("== .* == \n\n", content)[1:]) ]

In [51]:
with open("dene.txt") as fi:
    content = fi.read()
    result = re.split("== .* == \n\n", content)

result

['',
 "Şenol Gürşan, (d. 17 Ekim 1964, Pınarhisar, Kırklareli) Türk avukat ve siyasetçi.\nİstanbul Üniversitesi Hukuk Fakültesi'ni bitirmiş ve serbest avukat olarak çalışmıştır. Kırklareli İlim Yayma Cemiyeti Kuruculuğu ve Başkanlığı görevlerinde bulunmuştur.\n2009 yılında Adalet ve Kalkınma Partisi Kırklareli il yönetim kurulu üyesi olmuş, TBMM 24. dönem AK Parti Kırklareli milletvekili, Türkiye-Polonya Dostluk Grubu Başkanı ve TBMM KİT Komisyonu Sözcüsü olmuştur. Gelecek Partisi Kurucular Kurulu üyesi olup aynı zamanda partinin genel sekreteridir.\nİyi düzeyde Almanca bilen Gürşan, evli ve 2 çocuk babasıdır.\n\n",
 "Bovenau Almanya'nın kuzeyinde Schleswig-Holstein eyaletinde, Rendsburg-Eckernförde iline bağlı bölge. Rendsburg'un 10\xa0km. doğusunda yer alır. 26,2\xa0km² yüzölçümüne sahiptir. Nüfusu, 17 Temmuz 2006 itibarıyla yaklaşık 1098 olarak tespit edilmiştir. Belediye Başkanlığı Jürgen Liebsch tarafından yürütülmektedir."]

## Test For prepare create dataset

In [114]:
import data.tr_wiki67.prepare as prepare

prepare.create_and_save_tr_wiki_ds()

Saving the dataset (0/1 shards):   0%|          | 0/9989 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/383799 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/393794 [00:00<?, ? examples/s]